In [1]:
import datetime as dt
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np



In [5]:
WEEKDAY_NAMES_OPEN = ['Wednesday', 'Thursday', 'Sunday']
VOLUNTEERS_BY_WEEKDAY_NAME = {
    'Wednesday': ['Stephen', 'Sylvain'],
    'Thursday': ['Finlay', 'Sanna'],
    'Sunday': ['Alex M', '__'],
    }

def build_roster(year, month, weekday_names_open=WEEKDAY_NAMES_OPEN, 
  volunteers_by_weekday_name=VOLUNTEERS_BY_WEEKDAY_NAME):
    """
    INPUTS:
    
    - ``year``: integer
    - ``month``: integer 1--12
    
    
    OUTPUTS:
    
    A pandas data frame
    """
    d1 = dt.date(year, month, 1)
    d2 = d1 + relativedelta(months=1)
    ix = pd.date_range(d1, d2, closed='left')
    f = pd.DataFrame(index=ix)
    f['date'] = f.index.strftime('%Y-%m-%d')
    f['weekday'] = f.index.weekday_name
    
    # Assign volunteers
    f['volunteers'] = f['weekday'].map(lambda x:
      ' + '.join(volunteers_by_weekday_name.get(x, ["__", "__"])))
    
    # Restrict to given weekdays
    cond = f['weekday'].isin(weekday_names_open)
    f = f[cond].copy()
    
    # Add notes. 
    # Staff meeting is on the first Sunday of the month.
    f['notes'] = ''
    cond = f['weekday'] == 'Sunday'
    cond &= f.index.day <= 7
    f.loc[cond, 'notes'] = 'Staff meeting 15:00--16:00'
    
    return f.reset_index(drop=True)

roster = build_roster(2016, 8)
roster

,date,weekday,volunteers,notes
0,2016-08-03,Wednesday,Stephen + Sylvain,
1,2016-08-04,Thursday,Finlay + Sanna,
2,2016-08-07,Sunday,Alex M + __,Staff meeting 15:00--16:00
3,2016-08-10,Wednesday,Stephen + Sylvain,
4,2016-08-11,Thursday,Finlay + Sanna,
5,2016-08-14,Sunday,Alex M + __,
6,2016-08-17,Wednesday,Stephen + Sylvain,
7,2016-08-18,Thursday,Finlay + Sanna,
8,2016-08-21,Sunday,Alex M + __,
9,2016-08-24,Wednesday,Stephen + Sylvain,


In [6]:
def get_year_month(roster, format="%Y%m"):
    """
    Given a roster, return the year and month it pertains to 
    in the given format.
    """
    return pd.to_datetime(roster['date']).dt.strftime(format).iat[0]

def to_markdown(df, path):
    """
    Save the given data frame as a Markdown table located
    at the given path.
    """
    # Get column names
    cols = df.columns

    # Create a new DataFrame with just the markdown
    # strings
    df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)

    #Create a new concatenated DataFrame
    df3 = pd.concat([df2, df])

    #Save as markdown
    df3.to_csv(path, sep="|", index=False)


date = get_year_month(roster)
path = 'roster_{!s}.md'.format(date)
to_markdown(roster, path)